# Evidently Metrics 

In [ ]:
try:
    import evidently
except:
    !pip install git+https://github.com/evidentlyai/evidently.git

In [ ]:
import pandas as pd
import numpy as np

from sklearn import datasets
from sklearn import ensemble
from sklearn import model_selection

from evidently import ColumnMapping
from evidently.options import ColorOptions
from evidently.report import Report

from evidently.metrics import ColumnDriftMetric
from evidently.metrics import DataDriftTable
from evidently.metrics import DatasetDriftMetric
from evidently.metrics import ColumnCategoryMetric
from evidently.metrics import ColumnDistributionMetric
from evidently.metrics import ColumnValuePlot
from evidently.metrics import ColumnQuantileMetric
from evidently.metrics import ColumnCorrelationsMetric
from evidently.metrics import ColumnValueListMetric
from evidently.metrics import ColumnValueRangeMetric
from evidently.metrics import DatasetCorrelationsMetric
from evidently.metrics import ColumnRegExpMetric
from evidently.metrics import ColumnSummaryMetric
from evidently.metrics import ColumnMissingValuesMetric
from evidently.metrics import DatasetSummaryMetric
from evidently.metrics import DatasetMissingValuesMetric
from evidently.metrics import ConflictTargetMetric
from evidently.metrics import ConflictPredictionMetric
from evidently.metrics import ClassificationQualityMetric
from evidently.metrics import ClassificationClassBalance
from evidently.metrics import ClassificationConfusionMatrix
from evidently.metrics import ClassificationQualityByClass
from evidently.metrics import ClassificationClassSeparationPlot
from evidently.metrics import ClassificationProbDistribution
from evidently.metrics import ClassificationRocCurve
from evidently.metrics import ClassificationPRCurve
from evidently.metrics import ClassificationPRTable
from evidently.metrics import ClassificationQualityByFeatureTable
from evidently.metrics import ClassificationDummyMetric
from evidently.metrics import RegressionQualityMetric
from evidently.metrics import RegressionPredictedVsActualScatter
from evidently.metrics import RegressionPredictedVsActualPlot
from evidently.metrics import RegressionErrorPlot
from evidently.metrics import RegressionAbsPercentageErrorPlot
from evidently.metrics import RegressionErrorDistribution
from evidently.metrics import RegressionErrorNormality
from evidently.metrics import RegressionTopErrorMetric
from evidently.metrics import RegressionErrorBiasTable
from evidently.metrics import RegressionDummyMetric
#from evidently.metrics import EmbeddingsDriftMetric

## Prepare a Dataset

In [ ]:
#Dataset for Data Quality and Integrity
adult_data = datasets.fetch_openml(name='adult', version=2, as_frame='auto')
adult = adult_data.frame

adult_ref = adult[~adult.education.isin(['Some-college', 'HS-grad', 'Bachelors'])]
adult_cur = adult[adult.education.isin(['Some-college', 'HS-grad', 'Bachelors'])]

adult_cur.iloc[:2000, 3:5] = np.nan

In [ ]:
#Dataset for binary label and probabilistic classifcation
bcancer_data = datasets.load_breast_cancer(as_frame='auto')
bcancer = bcancer_data.frame

bcancer_ref = bcancer.sample(n=300, replace=False)
bcancer_cur = bcancer.sample(n=200, replace=False)

bcancer_label_ref = bcancer_ref.copy(deep=True)
bcancer_label_cur = bcancer_cur.copy(deep=True)

model = ensemble.RandomForestClassifier(random_state=1, n_estimators=10)
model.fit(bcancer_ref[bcancer_data.feature_names.tolist()], bcancer_ref.target)

bcancer_ref['prediction'] = model.predict_proba(bcancer_ref[bcancer_data.feature_names.tolist()])[:, 1]
bcancer_cur['prediction'] = model.predict_proba(bcancer_cur[bcancer_data.feature_names.tolist()])[:, 1]

bcancer_label_ref['prediction'] = model.predict(bcancer_label_ref[bcancer_data.feature_names.tolist()])
bcancer_label_cur['prediction'] = model.predict(bcancer_label_cur[bcancer_data.feature_names.tolist()])

In [ ]:
#Dataset for regression
housing_data = datasets.fetch_california_housing(as_frame='auto')
housing = housing_data.frame

housing.rename(columns={'MedHouseVal': 'target'}, inplace=True)
housing['prediction'] = housing_data['target'].values + np.random.normal(0, 3, housing.shape[0])

housing_ref = housing.sample(n=5000, replace=False)
housing_cur = housing.sample(n=5000, replace=False)

## How to run Reports?

### Data Drift Metrics

In [ ]:
#dataset-level metrics
data_drift_dataset_report = Report(metrics=[
    DataDriftTable(num_stattest='kl_div', cat_stattest='psi'),    
])

data_drift_dataset_report.run(reference_data=adult_ref, current_data=adult_cur)
data_drift_dataset_report

In [ ]:
#report in a JSON format
data_drift_dataset_report.json()

In [ ]:
#report as a python object
data_drift_dataset_report.as_dict()

In [ ]:
#column-level metrics
data_drift_column_report = Report(metrics=[
    ColumnDriftMetric('age'),
    ColumnValuePlot('age'),  
])

data_drift_column_report.run(reference_data=adult_ref, current_data=adult_cur)
data_drift_column_report

### Data Quality Metrics

In [ ]:
#dataset-level metrics
data_quality_dataset_report = Report(metrics=[
    DatasetCorrelationsMetric(),
    
])

data_quality_dataset_report.run(reference_data=adult_ref, current_data=adult_cur)
data_quality_dataset_report

In [ ]:
#column-level metrics
data_quality_column_report = Report(metrics=[
    ColumnDistributionMetric(column_name="education"), 
    ColumnQuantileMetric(column_name="education-num", quantile=0.75), 
    ColumnCorrelationsMetric(column_name="education"),
    ColumnValueListMetric(column_name="relationship", values=["Husband", "Unmarried"]), 
    ColumnValueRangeMetric(column_name="age", left=10, right=20),
    ColumnCategoryMetric(column_name='education', category='Some-college'),
    
])

data_quality_column_report.run(reference_data=adult_ref, current_data=adult_cur)
data_quality_column_report

### Data Integrity Metrics

In [ ]:
#dataset-level metrics
data_integrity_dataset_report = Report(metrics=[
    DatasetSummaryMetric(),
    DatasetMissingValuesMetric()
    
])

data_integrity_dataset_report.run(reference_data=adult_ref, current_data=adult_cur)
data_integrity_dataset_report

In [ ]:
#column-level metrics
data_integrity_column_report = Report(metrics=[
    ColumnRegExpMetric(column_name="relationship", reg_exp=r".*child.*"),
    ColumnSummaryMetric(column_name="age"),
    ColumnMissingValuesMetric(column_name="education"),

    
])

data_integrity_column_report.run(reference_data=adult_ref, current_data=adult_cur)
data_integrity_column_report

### Classification Metrics

In [ ]:
#label binary classification
classification_report = Report(metrics=[
    ClassificationQualityMetric(),
    ClassificationClassBalance(),
    ConflictTargetMetric(),
    ConflictPredictionMetric(),
    ClassificationConfusionMatrix(),
    ClassificationQualityByClass(),
    ClassificationDummyMetric(),
    ClassificationQualityByFeatureTable(columns=['mean area', 'fractal dimension error']),
])

classification_report.run(reference_data=bcancer_label_ref, current_data=bcancer_label_cur)
classification_report

In [ ]:
#probabilistic binary classification
classification_report = Report(metrics=[
    ClassificationQualityMetric(),
    ClassificationClassBalance(),
    ConflictTargetMetric(),
    ConflictPredictionMetric(),
    ClassificationConfusionMatrix(),
    ClassificationQualityByClass(),
    ClassificationClassSeparationPlot(),
    ClassificationProbDistribution(),
    ClassificationRocCurve(),
    ClassificationPRCurve(),
    ClassificationPRTable(),
    ClassificationQualityByFeatureTable(columns=['mean area', 'fractal dimension error']),

    
])

classification_report.run(reference_data=bcancer_ref, current_data=bcancer_cur)
classification_report

### Regreission Metrics

In [ ]:
regression_report = Report(metrics=[
    RegressionQualityMetric(),
    RegressionPredictedVsActualScatter(),
    RegressionPredictedVsActualPlot(),
    RegressionErrorPlot(),
    RegressionAbsPercentageErrorPlot(),
    RegressionErrorDistribution(),
    RegressionErrorNormality(),
    RegressionTopErrorMetric(),
    RegressionErrorBiasTable(columns=['MedInc', 'AveRooms']),
    RegressionDummyMetric(),
    ConflictTargetMetric(),
    ConflictPredictionMetric(),
    
])

regression_report.run(reference_data=housing_ref, current_data=housing_cur)
regression_report

### How to set metric parameters?

In [ ]:
#simple metric parameters
data_integrity_column_report = Report(metrics=[
    ColumnRegExpMetric(column_name="education", reg_exp=r".*-.*", top=5),
    ColumnRegExpMetric(column_name="relationship", reg_exp=r".*child.*")
])

data_integrity_column_report.run(reference_data=adult_ref, current_data=adult_cur)
data_integrity_column_report

In [ ]:
#options
color_scheme = ColorOptions(
    primary_color="#5a86ad",
    fill_color="#fff4f2",
    zero_line_color="#016795",
    current_data_color= "#c292a1" ,
    reference_data_color="#017b92",
)

In [ ]:
data_drift_column_report = Report(metrics=[
    ColumnDriftMetric('age'),
    ColumnDriftMetric('age', stattest='psi'),
],
                                  options=[color_scheme]
)

data_drift_column_report.run(reference_data=adult_ref, current_data=adult_cur)
data_drift_column_report

# Support Evidently
Did you find the example useful? Star Evidently on GitHub to contribute back! This helps us continue creating free open-source tools for the community. https://github.com/evidentlyai/evidently